In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import math
import gc

from sklearn.metrics import roc_auc_score

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data

### Data

In [9]:
df_1min = load_pkl('./experiments/feat_engin/data/data_shift_time_1min.pkl')
df_5min = load_pkl('./experiments/feat_engin/data/data_shift_time_5min.pkl')
df_1hour = load_pkl('./experiments/feat_engin/data/data_shift_time_1hour.pkl')
df_1day = load_pkl('./experiments/feat_engin/data/data_shift_time_1day.pkl')

In [155]:
np.where(df_1min['ticker'] == 'SNGS')[0][85831]

2404834

### Check "target""

In [215]:
i = 5898765

ind_5min = int(df_1min.loc[i, 'index_5min'])
ind_1hour = int(df_1min.loc[i, 'index_1hour'])
ind_1day = int(df_1min.loc[i, 'index_1day'])

cols = ['time', 'close', 'index_5min', 'index_1hour', 'index_1day', 'ticker', 'result', 'delta_time', 'res_price', 'res_ind']
df_1min[cols].iloc[i:i+22]



,time,close,index_5min,index_1hour,index_1day,ticker,result,delta_time,res_price,res_ind
index,,,,,,,,,,
5898765,2024-10-07 12:20:00,54.060001,1212263.0,104288.0,7335.0,ALRS,WIN,0 days 00:32:00,54.369999,5898797.0
5898766,2024-10-07 12:21:00,54.139999,1212263.0,104288.0,7335.0,ALRS,DNF,100 days 00:00:00,-1.000000,-1.0
5898767,2024-10-07 12:22:00,54.110001,1212263.0,104288.0,7335.0,ALRS,DNF,100 days 00:00:00,-1.000000,-1.0
5898768,2024-10-07 12:23:00,54.139999,1212263.0,104288.0,7335.0,ALRS,DNF,100 days 00:00:00,-1.000000,-1.0
5898769,2024-10-07 12:24:00,54.110001,1212263.0,104288.0,7335.0,ALRS,DNF,100 days 00:00:00,-1.000000,-1.0
5898770,2024-10-07 12:25:00,54.049999,1212264.0,104288.0,7335.0,ALRS,WIN,0 days 00:27:00,54.369999,5898797.0
5898771,2024-10-07 12:26:00,54.080002,1212264.0,104288.0,7335.0,ALRS,WIN,0 days 00:26:00,54.369999,5898797.0
5898772,2024-10-07 12:27:00,54.119999,1212264.0,104288.0,7335.0,ALRS,DNF,100 days 00:00:00,-1.000000,-1.0
5898773,2024-10-07 12:28:00,54.169998,1212264.0,104288.0,7335.0,ALRS,DNF,100 days 00:00:00,-1.000000,-1.0


In [218]:
df_1min['close'].iloc[i], df_1min['result'].iloc[i]

(54.06, 'WIN')

In [220]:
df_1min['close'].iloc[i:i+61].min(), df_1min['close'].iloc[i:i+61].max()

(54.04999923706055, 54.369998931884766)

In [222]:
df_1min['close'].iloc[i:i+61].min() < df_1min['close'].iloc[i]*0.995, df_1min['close'].iloc[i:i+61].max() > df_1min['close'].iloc[i]*1.005, 

(False, True)

### Compare "close"

In [227]:
cols_1min = ['time', 'close']
df_1min[cols_1min].iloc[i:i+1]

,time,close
index,,
5898765,2024-10-07 12:20:00,54.060001


In [229]:
cols_5min = ['time', 'close_5min']
df_5min[cols_5min].iloc[ind_5min:ind_5min+1]

,time,close_5min
index,,
1212263,2024-10-07 12:20:00,54.060001


In [231]:
cols_1hour = ['time', 'close_1hour']
df_1hour[cols_1hour].iloc[ind_1hour:ind_1hour+1]

,time,close_1hour
index,,
104288,2024-10-07 12:00:00,54.279999


In [233]:
cols_1day = ['time', 'close_1day']
df_1day[cols_1day].iloc[ind_1day:ind_1day+1]

,time,close_1day
index,,
7335,2024-10-04 23:50:00,54.52
